### 과제_1122_1
CIFAR10 데이터셋으로 GAN 모델을 수행하신 후 생성된 이미지 8개를 출력하세요

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Reshape, Conv2D, Conv2DTranspose, Flatten,Dropout
from tensorflow.keras.models import Model
import numpy as np

latent_dim = 32
height = 32
width = 32
channels = 3
generator_input = Input(shape=(latent_dim,))

x = Dense(128*16*16)(generator_input)
x = LeakyReLU()(x)
x = Reshape((16,16,128))(x)

x = Conv2D(256,5,padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(256,4,strides=2,padding='same')(x)
x = LeakyReLU()(x)

x = Conv2D(256,5,padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(256,5,padding='same')(x)
x = LeakyReLU()(x)

x = Conv2D(channels,7,activation='tanh',padding='same')(x)
generator = Model(generator_input,x)
generator.summary()


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_4 (Dense)             (None, 32768)             1081344   
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 32768)             0         
                                                                 
 reshape_2 (Reshape)         (None, 16, 16, 128)       0         
                                                                 
 conv2d_18 (Conv2D)          (None, 16, 16, 256)       819456    
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 256)      1048

In [10]:
from tensorflow.keras.optimizers import RMSprop
discriminator_input = Input(shape=(height, width, channels))
x = Conv2D(128,3)(discriminator_input)
x = LeakyReLU()(x)
x = Conv2D(128,4,strides=2)(x)
x = LeakyReLU()(x)
x = Conv2D(128,4,strides=2)(x)
x = LeakyReLU()(x)
x = Conv2D(128,4,strides=2)(x)
x = LeakyReLU()(x)
x = Flatten()(x)

x = Dropout(0.4)(x)

x = Dense(1,activation='sigmoid')(x)

discriminator = Model(discriminator_input,x)
discriminator.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 30, 30, 128)       3584      
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 30, 30, 128)       0         
                                                                 
 conv2d_23 (Conv2D)          (None, 14, 14, 128)       262272    
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 14, 14, 128)       0         
                                                                 
 conv2d_24 (Conv2D)          (None, 6, 6, 128)         262272    
                                                                 
 leaky_re_lu_25 (LeakyReLU)  (None, 6, 6, 128)         0   

In [11]:
discriminator_optimizer = RMSprop(lr=0.0008, clipvalue=1.0,decay=1e-8) # 모든 매개변수 경사를 최대값 1.0와 최소값 -1.0으로 자른다.
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [12]:
discriminator.trainable = False

gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input,gan_output)

gan_optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss= 'binary_crossentropy')

In [ ]:
import os
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
(x_train,y_train),(_,_) = cifar10.load_data()

x_train = x_train[y_train.flatten() == 6]

x_train = x_train.reshape(
    (x_train.shape[0],) + (height,width,channels)).astype('float32') / 255. 

iteration = 10000
batch_size = 20
save_dir = './gan_images/'
if not os.path.exists(save_dir):
  os.mkdir(save_dir)


170498071/170498071 [==============================] - 11s 0us/step
